In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from surprise import Dataset, Reader, SVD

In [2]:
df = pd.read_csv('/home/longnguyen/Documents/Coding/FDE-Course-2024/Project-Day/Youtube-ETL-Pipeline/dataset/youTube_trending_video/DE_youtube_trending_data.csv')
df.head()

,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled
0,KJi2qg5F-9E,Bonez MC - HOLLYWOOD (Snippet),2020-08-11T18:00:03Z,UCGh8tmH9x9njaI2mXfh2fyg,CrhymeTV,10,2020-08-12T00:00:00Z,187|187 Strassenbande|BONEZ MC|RAF Camora|MAXW...,573902,69319,970,3311,https://i.ytimg.com/vi/KJi2qg5F-9E/default.jpg,False,False
1,K0vYnOn7wZI,Nik hat heftige Probleme in Köln! 😱😫 #1925 | K...,2020-08-11T17:00:09Z,UCnrvUg5MJWPDSrv_voT7AqA,Köln 50667,24,2020-08-12T00:00:00Z,Köln|50667|RTL2|RTL|Serie|k50667|ganze folge|t...,381375,13637,435,866,https://i.ytimg.com/vi/K0vYnOn7wZI/default.jpg,False,False
2,2bbn9b79LRc,Camper Tour 2020 - ROADTRIP durch Österreich |...,2020-08-11T18:00:00Z,UCBt8RY61tvanrhkzeZdNICw,AnaJohnson,24,2020-08-12T00:00:00Z,camper tour|camper tour deutsch|camper tour de...,142296,9480,144,364,https://i.ytimg.com/vi/2bbn9b79LRc/default.jpg,False,False
3,Zv-3qNnAMaM,Ich TESTE SHEIN BIKINIS (try on haul) - UNMÖGL...,2020-08-12T09:00:08Z,UCccDoH6QpRCjjcMgl5f88wA,Einfach Marci,24,2020-08-12T00:00:00Z,[None],55640,3420,124,229,https://i.ytimg.com/vi/Zv-3qNnAMaM/default.jpg,False,False
4,7clgQLneouU,STATEMENT zu meinem BAN! 😒 | Bald vielleicht P...,2020-08-11T12:00:02Z,UC8E8eD7mOcnMazJT4laKbFQ,Domo,24,2020-08-12T00:00:00Z,Papaplatte|papaplatte twitch best of|papaplatt...,233899,25251,375,1051,https://i.ytimg.com/vi/7clgQLneouU/default.jpg,False,False


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 268736 entries, 0 to 268735
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   video_id           268736 non-null  object
 1   title              268736 non-null  object
 2   publishedAt        268736 non-null  object
 3   channelId          268736 non-null  object
 4   channelTitle       268736 non-null  object
 5   categoryId         268736 non-null  int64 
 6   trending_date      268736 non-null  object
 7   tags               268736 non-null  object
 8   view_count         268736 non-null  int64 
 9   likes              268736 non-null  int64 
 10  dislikes           268736 non-null  int64 
 11  comment_count      268736 non-null  int64 
 12  thumbnail_link     268736 non-null  object
 13  comments_disabled  268736 non-null  bool  
 14  ratings_disabled   268736 non-null  bool  
dtypes: bool(2), int64(5), object(8)
memory usage: 27.2+ MB


In [4]:
content = df.drop_duplicates(subset=['video_id'],keep='last')

In [5]:
content.shape

(55476, 15)

In [6]:
def fill_mi(v):
    if v == 1: return 'Film & Animation'
    elif v == 2: return 'Autos & Vehicles'
    elif v == 10: return 'Music'
    elif v == 15: return 'Pets & Animals'
    elif v == 17: return 'Sports'
    elif v == 19: return 'Travel & Events'
    elif v == 20: return 'Gaming'
    elif v == 22: return 'People & Blogs'
    elif v == 23: return 'Comedy'
    elif v == 24: return 'Entertainment'
    elif v == 25: return 'News & Politics'
    elif v == 26: return 'Howto & Style'
    elif v == 27: return 'Education'
    elif v == 28: return 'Science & Technology'
    else: return 'Nonprofits & Activism'
        
        

In [7]:
content['categoryname'] = content['categoryId'].apply(fill_mi)

/tmp/ipykernel_57859/3480152712.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content['categoryname'] = content['categoryId'].apply(fill_mi)


In [8]:
len(content['categoryname'])

55476

In [9]:
content['content'] = content[['title', 'channelId', 'channelTitle', 'categoryname']].apply(lambda row: " ".join(row.dropna().astype(str)), axis=1)
tfidf_vectorizer = TfidfVectorizer()
content_matrix = tfidf_vectorizer.fit_transform(content['content'])

/tmp/ipykernel_57859/1319113376.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content['content'] = content[['title', 'channelId', 'channelTitle', 'categoryname']].apply(lambda row: " ".join(row.dropna().astype(str)), axis=1)


In [10]:
content_similarity = linear_kernel(content_matrix, content_matrix)

MemoryError: Unable to allocate 22.9 GiB for an array with shape (55476, 55476) and data type float64

: 

In [ ]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(content[['video_id', 'view', 'like', 'dislike', 'comment', 'categoryid']], reader)
